In [22]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from matplotlib import pyplot as plt

In [23]:
NUM_NEURON = 10
NUM_FEATURES = 7

learning_rate = 10**-3
beta = 10**-3

epochs = 1500
batch_size = 8

df = pd.read_csv("data/admission_predict.csv")
df = df.drop(['Serial No.'], axis=1)

y = df['Chance of Admit'].to_numpy()
y = np.reshape(y, (y.shape[0], 1))


#Optimized one 
features = df.drop(["Chance of Admit"], axis=1).to_numpy()

In [24]:
def neural_net(x):
    #hidden layer 1
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1) #activation function
    
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return (out_layer)


weights = {
    'h1': tf.Variable(tf.random_normal([NUM_FEATURES, NUM_NEURON], seed=1337)),
    'out': tf.Variable(tf.random_normal([NUM_NEURON, 1], seed=1337))# 1 ouput label
}

biases = {
    'b1': tf.Variable(tf.random_normal([10], seed=1337)),
    'out': tf.Variable(tf.random_normal([1], seed=1337))
}

In [25]:
np.random.seed(1337)
tf.set_random_seed(1337)

#Create model
X = tf.placeholder(tf.float32, [None, NUM_FEATURES])
Y = tf.placeholder(tf.float32, [None, 1])


x_train, x_test, y_train, y_test = train_test_split(features, y, test_size=0.3)
scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#predicted value
Y_hat = neural_net(X)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
loss = tf.reduce_mean(tf.square(Y - Y_hat)) 
reg = tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])
train_op = optimizer.minimize(loss + reg * beta)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_train_err = []
    test_err = []
    for i in range(epochs):

        #Batch training
        #pick random batch_size data points from x_train
        rand_index = np.random.choice(x_train.shape[0], size=batch_size)
        x_batch = x_train[rand_index]
        y_batch = y_train[rand_index]

        train_op.run(feed_dict={X:x_batch, Y:y_batch})
        batch_train_err.append(loss.eval(feed_dict={X: x_batch, Y: y_batch}))
        test_err.append(loss.eval(feed_dict={X: x_test, Y: y_test}))   


    sample_index = np.random.choice(x_test.shape[0], size=50)
    x_test_sample = x_test[sample_index]
    y_test_sample = y_test[sample_index]
    #plot y_hat and y_test

    pred = sess.run(Y_hat, feed_dict={X:x_test_sample})

print("Batch Train Err " + str(NUM_FEATURES) + " features:")
print (batch_train_err[-1])
print (test_err[-1])

Batch Train Err 7 features:
0.29753163
0.41722623
